# Contents:

1. Analyse point clouds, taking **ground detection** as an example
2. Introduce **organised point cloud**

## 1. Coordinate System

In [1]:
# imports
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Read point cloud
pcd = o3d.io.read_point_cloud("./data/depth_2_pcd.ply")

# Create a 3D coordinate system
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# geometries to draw
geometries = [pcd, origin]

# visualize
o3d.visualization.draw_geometries(geometries)

The blue, the red and the green arrows represent the Z-axis, the X-axis and the Y-axis respectively

In [4]:
# Get max and min x,y,z coordinates of the point cloud
x_max = max(pcd.points, key=lambda x:x[0])
y_max = max(pcd.points, key=lambda x:x[1])
z_max = max(pcd.points, key=lambda x:x[2])
x_min = min(pcd.points, key=lambda x:x[0])
y_min = min(pcd.points, key=lambda x:x[1])
z_min = min(pcd.points, key=lambda x:x[2])

In [8]:
# Colors
RED = [1., 0., 0.]
GREEN = [0., 1., 0.]
BLUE = [0., 0., 1.]
YELLOW = [1., 1., 0.]
MAGENTA = [1., 0., 1.]
CYAN = [1., 1., 1.]

positions = [x_max, y_max, z_max, x_min, y_min, z_min]
colors = [RED, GREEN, BLUE, YELLOW, MAGENTA, CYAN]

for i in range(len(positions)):
    # create a sphere mesh
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)

    # move to the point position
    sphere.translate(np.asarray(positions[i]))

    # add color
    sphere.paint_uniform_color(np.asarray(colors[i]))

    # compute normals for vertices or faces
    sphere.compute_vertex_normals()

    # add to geometry list to display later
    geometries.append(sphere)

o3d.visualization.draw_geometries(geometries)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
